![banner](/custom/banner.png)


# FSLeyes Jupyter notebook


## Help


Use the built-in `help` function to get help on anything in the shell
environment, whether it is a module, function, class, or object:

    help(load)



## Overlays


The `overlayList` is a list which contains all of the overlays that have been
loaded into FSLeyes.  You can load overlays by using the `load` function,
e.g.:

    std = load('MNI152_T1_2mm.nii.gz')


You can remove an overlay simply by removing it from the `overlayList`:

    del overlayList[0]


## Display settings


You can use the `displayCtx` to access the display settings for an
overlay. Display settings for an overlay are divided between two objects:

 - A `Display` object, which contains general display settings
 - A `DisplayOpts` object, which contains display settings specific to the
   overlay type


You can access these objects like so:

    overlay = overlayList[0]
    display = displayCtx.getDisplay(overlay)
    opts    = displayCtx.getOpts(   overlay)


Then adjusting the overlay display settings is easy:


    display.brightness = 75
    opts.cmap          = 'hot'


## FSL commands


Many FSL commands can be called through Python functions, e.g.:

    flirt('src.nii.gz', 'ref.nii.gz', applyxfm=True, init='src2ref.mat')


For commands which produce output images, you can have the outputs loaded
directly into FSLeyes by using the special `LOAD` symbol:

    struc = Image('struct.nii.gz')
    bet(struc, LOAD)


To submit a command as a cluster job, use the `submit` flag:

    fnirt('src.nii.gz', 'ref.nii.gz', submit=True)


## User interface


You can access the current views in FSLeyes through the ``frame`` object:

    ortho = frame.viewPanels[0]


You also can add/remove views through the ``frame``:


    frame.removeViewPanel(ortho)
    frame.addViewPanel(Scene3DPanel)


## Screenshots


To insert a screenshot of the currently displayed scene into the notebook, use
the `screenshot` function:

    ortho = frame.viewPanels[0]
    screenshot(ortho)


## Reproducing a scene


The `getState` function returns a string which describes the current FSLeyes
state. This string describes the current FSLeyes user interface (views,
controls, and toolbars) as well as all loaded overlays and their display
settings.


You can also use the `setState` function to reproduce a previously generated
FSLeyes state.


To make your notebook reproduce your FSLeyes setup, set up your user interface
and overlays how you want, and then call `getState` to get a description of
the current FSLeyes setup:


    getState()


Now copy the string that `getState` generates, and then at the top of your
notebook add a call to `setState`, passing in that string, e.g.:

    fsleyesState = 'OrthoPanel\nlayout2|name=OrthoPanel 1;caption=...'
    setState(fsleyesState)


## For more information

See the FSLeyes and `fslpy` API documentation:

 - https://open.win.ox.ac.uk/pages/fsl/fsleyes/fsleyes/apidoc/
 - https://open.win.ox.ac.uk/pages/fsl/fslpy/

In [1]:
overlayList

[Image(MNI152_T1_2mm, /Users/xpsy1114/fsl/data/standard/MNI152_T1_2mm.nii.gz)]

In [2]:
std = overlayList[0]
bet(std, output=LOAD)


{'output': Image(output, None)}

In [3]:
overlayList

[Image(MNI152_T1_2mm, /Users/xpsy1114/fsl/data/standard/MNI152_T1_2mm.nii.gz), Image(output, None)]

In [4]:
overlay = overlayList[1]

display       = displayCtx.getDisplay(overlay)
display.alpha = 50

opts      = displayCtx.getOpts(   overlay)
opts.cmap = 'red-yellow'

In [5]:
# Import a useful resampling function
from fsl.utils.image.resample import resampleToPixdims

# the tensor is the latest overlay:
tensor_1mm = overlayList[-1]

# resampling outputs a numpy array as well as the new xform to put in the header for display in FSLeyes
tensor_2mm, xform = resampleToPixdims(tensor_1mm, [2,2,2,1])

# we need to add the resampled tensor to the overlay list. 
# we can't just add a numpy array, it needs to be an "Image" object, with the correct xform:
overlayList.append(Image(tensor_2mm, xform=xform), name='tensor2mm', cmap='grey')

# we can delete the 1mm tensor:
del overlayList[-2]

In [6]:
# short way of getting the 6 tensor elements
# the transpose just puts the 4th dimension first so we can 
# get the separate 3D files as if it were a list
Dxx, Dxy, Dxz, Dyy, Dyz, Dzz = np.transpose(tensor_2mm,(3,0,1,2))
# take the mean of the diagonal elements
MD = (Dxx+Dyy+Dzz)/3.
# add to the overlay list
overlayList.append(Image(MD, xform=xform), name='MD', cmap='grey')

In [7]:
# fslmaths works a little bit like command line interface
# you keep appending options and run .run() at the end
MD = fslmaths(Image(Dxx)).add(Image(Dyy)).add(Image(Dzz)).div(3.).run()
# the result of fslmaths is an Image object but with the wrong header
# below we just get the data as numpy array and use that to create a new object 
# to add to the overlay list with the correct header
MD = MD.data
overlayList.append(Image(MD, xform=xform), name='MD_fslmaths', cmap='grey')

In [8]:
L1, L2, L3 = np.zeros_like(MD), np.zeros_like(MD), np.zeros_like(MD)
mask = MD>0
for x in range(tensor_2mm.shape[0]):
    for y in range(tensor_2mm.shape[1]):
        for z in range(tensor_2mm.shape[2]):
            if mask[x,y,z]:
                tens = np.array([[Dxx[x,y,z], Dxy[x,y,z], Dxz[x,y,z]],
                                 [Dxy[x,y,z], Dyy[x,y,z], Dyz[x,y,z]],
                                 [Dxz[x,y,z], Dyz[x,y,z], Dzz[x,y,z]]])
                L, _ = np.linalg.eigh(tens)
                L1[x,y,z] = L[2]
                L2[x,y,z] = L[1]
                L3[x,y,z] = L[0]

In [9]:
numer = np.sqrt((L1-MD)**2+(L2-MD)**2+(L3-MD)**2)
denom   = np.sqrt(L1**2+L2**2+L3**2)
FA    = numer/denom*np.sqrt(3/2)

# remove nans from outside the brain mask
FA[np.isnan(FA)] = 0.
# add to the list
overlayList.append(Image(FA, xform=xform), name='FA', cmap='grey')

In [10]:
opts.cmap = 'red-yellow'